In [ ]:
import os
import cv2
import numpy as np

from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import albumentations as A
import albumentations.augmentations.functional as F
from albumentations.pytorch import ToTensorV2

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive

# Google Colab with Personal Google Drive
drive.mount('/content/drive')
# Change to project folder
path = r"/content/drive/MyDrive/COMP6200 Master Project"
os.chdir(path)
os.path.abspath(os.curdir)

Mounted at /content/drive


'/content/drive/MyDrive/COMP6200 Master Project'

# Image Segmentation

image size: width x height

NIR Images：\
'CASIA-Iris-Africa': 1088x640\
Resize: 512x512\
After split:\
train_images_len = 296(80%)\
val_images_len = 74(20%)\
africa_verify_transform = A.Normalize(mean=(0.31808122, 0.31808122, 0.31808122), std=(0.13381795, 0.13381795, 0.13381795))

'CASIA-Iris-Asia/CASIA-distance': 640x480
Resize: 512x512\
After split:\
train_images_len = 236(80%)\
val_images_len = 60(20%)\
distance_verify_transform = A.Normalize(mean=(0.65741013, 0.65741013, 0.65741013), std=(0.17129371, 0.17129371, 0.17129371))

'CASIA-Iris-Asia/CASIA-Iris-Complex/Occlusion': 640x480\
Resize: 512x512\
After split:\
train_images_len = 320(80%)\
val_images_len = 80(20%)\
occlusion_verify_transform = A.Normalize(mean=(0.55987521, 0.55271507, 0.55786177), std=(0.14799709, 0.14656123, 0.14819625))


'CASIA-Iris-Asia/CASIA-Iris-Complex/Off_angle': 640x480\
Resize: 512x512\
After split:\
train_images_len = 320(80%)\
val_images_len = 80(20%)\
off_angle_verify_transform = A.Normalize(mean=(0.57691509, 0.57053768, 0.57574402), std=(0.13347985, 0.13234827, 0.13389373))


'CASIA-Iris-Asia/CASIA-Iris-M1': 400x400\
Resize: 512x512\
After split:\
train_images_len = 1200(80%)\
val_images_len = 300(20%)\
m1_verify_transform = A.Normalize(mean=(0.56574345, 0.56574345, 0.56574345), std=(0.20760348, 0.20760348, 0.20760348))


Visible light：\
'UBIRIS.v2_seg': 不规则\
Resize: 512x512\
After split:\
train_images_len = 1356(80%)\
val_images_len = 339(20%)\
UBIRIS_v2_seg_verify_transform = A.Normalize(mean=(0.30213674, 0.35349771, 0.49928186), std=(0.14017686, 0.14544913, 0.17964584))



'MICHE_seg': 400x300\
Resize: 512x512\
After split:\
train_images_len = 544(80%)\
val_images_len = 136(20%)\
MICHE_seg_verify_transform = A.Normalize(mean=(0.35226458, 0.39623704, 0.52841291), std=(0.1711761,  0.18303123, 0.20146306))



'all_datasets':\
Resize: 512x512\
After split:\
train_images_len = 4272\
all_datasets_verify_transform = A.Normalize(mean=(0.44059682, 0.46168045, 0.52601482), std=(0.16324673, 0.16629836, 0.18005962))


In [ ]:
def binary_the_mask(mask_path):
  mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
  _, mask_binary = cv2.threshold(mask, 127, 1, cv2.THRESH_BINARY)
  return mask_binary

class DatasetSeg(Dataset):

    def __init__(self, root, dataset_name, transform=None):
        self.root = root
        self.dataset_name = dataset_name
        self.images_dir = 'image'
        self.masks_dir = 'SegmentationClass'

        self.images_path = os.path.join(self.root, self.images_dir)
        self.masks_path = os.path.join(self.root, self.masks_dir)
        self.images_list = list(os.listdir(self.images_path))
        self.masks_list = list(os.listdir(self.masks_path))

        self.transform = transform


    def __len__(self):
      return len(self.images_list) # how many pictures
  
    def __getitem__(self, idx):
        image_filename = self.images_list[idx].split('.')[0]

        if self.dataset_name == 'CASIA-Iris-Africa' or self.dataset_name == 'CASIA-Iris-Asia/CASIA-distance':
          image = cv2.imread(os.path.join(self.images_path, (image_filename + '.JPEG')), cv2.IMREAD_UNCHANGED)
          mask = cv2.imread(os.path.join(self.masks_path, (image_filename +'.png')), cv2.IMREAD_UNCHANGED)
          mask_binary = binary_the_mask(os.path.join(self.masks_path, (image_filename +'.png')))

        if self.dataset_name == 'CASIA-Iris-Asia/CASIA-Iris-Complex/Occlusion' or self.dataset_name == 'CASIA-Iris-Asia/CASIA-Iris-Complex/Off_angle':
          image = cv2.imread(os.path.join(self.images_path, (image_filename + '.jpg')), cv2.IMREAD_UNCHANGED)
          mask = cv2.imread(os.path.join(self.masks_path, (image_filename +'.png')), cv2.IMREAD_UNCHANGED)
          mask_binary = binary_the_mask(os.path.join(self.masks_path, (image_filename +'.png')))

        if self.dataset_name == 'CASIA-Iris-Asia/CASIA-Iris-M1':
          image = cv2.imread(os.path.join(self.images_path, (image_filename + '.JPG')), cv2.IMREAD_UNCHANGED)
          mask = cv2.imread(os.path.join(self.masks_path, (image_filename +'.png')), cv2.IMREAD_UNCHANGED)
          mask_binary = binary_the_mask(os.path.join(self.masks_path, (image_filename +'.png')))

        if self.dataset_name == 'UBIRIS_v2_seg':
          image = cv2.imread(os.path.join(self.images_path, (image_filename + '.tiff')), cv2.IMREAD_UNCHANGED)
          mask = cv2.imread(os.path.join(self.masks_path, (image_filename +'.tiff')), cv2.IMREAD_UNCHANGED)
          mask_binary = binary_the_mask(os.path.join(self.masks_path, (image_filename +'.tiff')))

        if self.dataset_name =='MICHE_seg':
          image = cv2.imread(os.path.join(self.images_path, (image_filename + '.JPEG')), cv2.IMREAD_UNCHANGED)
          mask = cv2.imread(os.path.join(self.masks_path, (image_filename +'.png')), cv2.IMREAD_UNCHANGED)
          mask_binary = binary_the_mask(os.path.join(self.masks_path, (image_filename +'.png')))

        if self.transform is not None:
          augmentation = self.transform(image=image, mask=mask_binary)
          image_aug = augmentation['image']
          mask_binary_aug = augmentation['mask']
          return image_aug, mask_binary_aug

        return image, mask_binary

In [ ]:
mobilevit_resize_transform = A.Compose(
    [
        A.Resize(512, 512, interpolation=cv2.INTER_LINEAR),
    ]
)


CASIA_Iris_Africa_train = DatasetSeg('./data/CASIA-Iris-Africa/train', 'CASIA-Iris-Africa', transform=mobilevit_resize_transform)
CASIA_distance_train = DatasetSeg('./data/CASIA-Iris-Asia/CASIA-distance/train', 'CASIA-Iris-Asia/CASIA-distance', transform=mobilevit_resize_transform)
CASIA_Iris_Complex_Occlusion_train = DatasetSeg('./data/CASIA-Iris-Asia/CASIA-Iris-Complex/Occlusion/train', 'CASIA-Iris-Asia/CASIA-Iris-Complex/Occlusion', transform=mobilevit_resize_transform)
CASIA_Iris_Complex_Off_angle_train = DatasetSeg('./data/CASIA-Iris-Asia/CASIA-Iris-Complex/Off_angle/train', 'CASIA-Iris-Asia/CASIA-Iris-Complex/Off_angle', transform=mobilevit_resize_transform)
CASIA_Iris_M1_train = DatasetSeg('./data/CASIA-Iris-Asia/CASIA-Iris-M1/train', 'CASIA-Iris-Asia/CASIA-Iris-M1', transform=mobilevit_resize_transform)

UBIRIS_v2_seg_train = DatasetSeg('./data/UBIRIS_v2_seg/train', 'UBIRIS_v2_seg', transform=mobilevit_resize_transform)
MICHE_seg_train = DatasetSeg('./data/MICHE_seg/train', 'MICHE_seg', transform=mobilevit_resize_transform)

In [ ]:
all_datasets_train = torch.utils.data.ConcatDataset([CASIA_Iris_Africa_train, 
                                                     CASIA_distance_train,
                                                     CASIA_Iris_Complex_Occlusion_train,
                                                     CASIA_Iris_Complex_Off_angle_train,
                                                     CASIA_Iris_M1_train,
                                                     UBIRIS_v2_seg_train,
                                                     MICHE_seg_train])

In [ ]:
def split_train_dataset(train_dataset_all, seed=2022):
  train_size= int(len(train_dataset_all)*0.8)
  val_size= int(len(train_dataset_all)-train_size)
  split_seed=seed
  train_dataset, val_dataset = torch.utils.data.random_split(train_dataset_all, [train_size, val_size], generator=torch.Generator().manual_seed(split_seed))
  
  return train_dataset, val_dataset

In [ ]:
CASIA_Iris_Africa_train_dataset, CASIA_Iris_Africa_val_dataset = split_train_dataset(CASIA_Iris_Africa_train)
CASIA_distance_train_dataset, CASIA_distance_val_dataset = split_train_dataset(CASIA_distance_train)
CASIA_Iris_Complex_Occlusion_train_dataset, CASIA_Iris_Complex_Occlusion_val_dataset = split_train_dataset(CASIA_Iris_Complex_Occlusion_train)
CASIA_Iris_Complex_Off_angle_train_dataset, CASIA_Iris_Complex_Off_angle_val_dataset = split_train_dataset(CASIA_Iris_Complex_Off_angle_train)
CASIA_Iris_M1_train_dataset, CASIA_Iris_M1_train_val_dataset = split_train_dataset(CASIA_Iris_M1_train)


UBIRIS_v2_seg_train_dataset, UBIRIS_v2_seg_val_dataset = split_train_dataset(UBIRIS_v2_seg_train)
MICHE_seg_train_dataset, MICHE_seg_M1_val_dataset = split_train_dataset(MICHE_seg_train)

all_datasets_train_dataset, all_datasets_train_val_dataset = split_train_dataset(all_datasets_train)

## Get Mean And Std Of Datasets

In [ ]:
def getMeanAndStd(train_dataset, channels):
    '''
    Compute mean and variance for training data
    :param train_data: Dataset
    :return: (mean, std)
    '''
    length = len(train_dataset)
    mean = np.zeros((channels,), dtype=np.float64)
    std = np.zeros((channels,), dtype=np.float64)

    for image, _ in tqdm(iter(train_dataset)):
        for d in range(channels):
            image_value = np.reshape(image[:,:,d], -1)
            mean[d] += np.mean(image_value)
            std[d] += np.std(image_value)
    mean = mean/length/255
    std = std/length/255
    return mean, std

In [ ]:
CASIA_Iris_Africa_train_dataset_mean, CASIA_Iris_Africa_train_dataset_std = getMeanAndStd(CASIA_Iris_Africa_train_dataset, 3)
print('\n')
print(CASIA_Iris_Africa_train_dataset_mean)
print(CASIA_Iris_Africa_train_dataset_std)

296it [00:12, 23.36it/s]



[0.31808122 0.31808122 0.31808122]
[0.13381795 0.13381795 0.13381795]


In [ ]:
CASIA_distance_train_dataset_mean, CASIA_distance_train_dataset_std = getMeanAndStd(CASIA_distance_train_dataset, 3)
print('\n')
print(CASIA_distance_train_dataset_mean)
print(CASIA_distance_train_dataset_std)

236it [00:06, 39.28it/s]



[0.65741013 0.65741013 0.65741013]
[0.17129371 0.17129371 0.17129371]


In [ ]:
CASIA_Iris_Complex_Occlusion_train_dataset_mean, CASIA_Iris_Complex_Occlusion_train_dataset_std = getMeanAndStd(CASIA_Iris_Complex_Occlusion_train_dataset, 3)
print('\n')
print(CASIA_Iris_Complex_Occlusion_train_dataset_mean)
print(CASIA_Iris_Complex_Occlusion_train_dataset_std)

320it [02:08,  2.49it/s]



[0.55987521 0.55271507 0.55786177]
[0.14799709 0.14656123 0.14819625]


In [ ]:
CASIA_Iris_Complex_Off_angle_train_dataset_mean, CASIA_Iris_Complex_Off_angle_train_dataset_std = getMeanAndStd(CASIA_Iris_Complex_Off_angle_train_dataset, 3)
print('\n')
print(CASIA_Iris_Complex_Off_angle_train_dataset_mean)
print(CASIA_Iris_Complex_Off_angle_train_dataset_std)

320it [02:03,  2.60it/s]



[0.57691509 0.57053768 0.57574402]
[0.13347985 0.13234827 0.13389373]


In [ ]:
CASIA_Iris_M1_train_dataset_mean, CASIA_Iris_M1_train_dataset_std = getMeanAndStd(CASIA_Iris_M1_train_dataset, 3)
print('\n')
print(CASIA_Iris_M1_train_dataset_mean)
print(CASIA_Iris_M1_train_dataset_std)

1200it [00:22, 53.17it/s]



[0.56574345 0.56574345 0.56574345]
[0.20760348 0.20760348 0.20760348]


In [ ]:
UBIRIS_v2_seg_train_dataset_mean, UBIRIS_v2_seg_train_dataset_std = getMeanAndStd(UBIRIS_v2_seg_train_dataset, 3)
print('\n')
print(UBIRIS_v2_seg_train_dataset_mean)
print(UBIRIS_v2_seg_train_dataset_std)

1356it [00:26, 51.74it/s]



[0.30213674 0.35349771 0.49928186]
[0.14017686 0.14544913 0.17964584]


In [ ]:
MICHE_seg_train_dataset_mean, MICHE_seg_train_dataset_std = getMeanAndStd(MICHE_seg_train_dataset, 3)
print('\n')
print(MICHE_seg_train_dataset_mean)
print(MICHE_seg_train_dataset_std)

544it [00:10, 50.02it/s]



[0.35226458 0.39623704 0.52841291]
[0.1711761  0.18303123 0.20146306]


In [ ]:
all_datasets_mean, all_datasets_std = getMeanAndStd(all_datasets_train_dataset, 3)
print('\n')
print(all_datasets_mean)
print(all_datasets_std)

4272it [34:12,  2.08it/s]



[0.44059682 0.46168045 0.52601482]
[0.16324673 0.16629836 0.18005962]


## Verify results

In [ ]:
africa_verify_transform = A.Normalize(mean=(0.31808122, 0.31808122, 0.31808122), std=(0.13381795, 0.13381795, 0.13381795))

distance_verify_transform = A.Normalize(mean=(0.65741013, 0.65741013, 0.65741013), std=(0.17129371, 0.17129371, 0.17129371))

occlusion_verify_transform = A.Normalize(mean=(0.55987521, 0.55271507, 0.55786177), std=(0.14799709, 0.14656123, 0.14819625))

off_angle_verify_transform = A.Normalize(mean=(0.57691509, 0.57053768, 0.57574402), std=(0.13347985, 0.13234827, 0.13389373))

m1_verify_transform = A.Normalize(mean=(0.56574345, 0.56574345, 0.56574345), std=(0.20760348, 0.20760348, 0.20760348))

UBIRIS_v2_seg_verify_transform = A.Normalize(mean=(0.30213674, 0.35349771, 0.49928186), std=(0.14017686, 0.14544913, 0.17964584))

MICHE_seg_verify_transform = A.Normalize(mean=(0.35226458, 0.39623704, 0.52841291), std=(0.1711761,  0.18303123, 0.20146306))

all_datasets_verify_transform = A.Normalize(mean=(0.44059682, 0.46168045, 0.52601482), std=(0.16324673, 0.16629836, 0.18005962))



In [ ]:
class DatasetFromSubset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, idx):
        image, mask_binary = self.subset[idx]

        if self.transform is not None:
          augmentation = self.transform(image=image, mask=mask_binary)
          image_aug = augmentation['image']
          mask_binary_aug = augmentation['mask']
          return image_aug, mask_binary_aug

    def __len__(self):
        return len(self.subset)

In [ ]:
africa_train_aftersplit = DatasetFromSubset(CASIA_Iris_Africa_train_dataset, transform=africa_verify_transform)
distance_train_aftersplit = DatasetFromSubset(CASIA_distance_train_dataset, transform=distance_verify_transform)
occ_train_aftersplit = DatasetFromSubset(CASIA_Iris_Complex_Occlusion_train_dataset, transform=occlusion_verify_transform)
off_train_aftersplit = DatasetFromSubset(CASIA_Iris_Complex_Off_angle_train_dataset, transform=off_angle_verify_transform)
m1_train_aftersplit = DatasetFromSubset(CASIA_Iris_M1_train_dataset, transform=m1_verify_transform)
ubiris_train_aftersplit = DatasetFromSubset(UBIRIS_v2_seg_train_dataset, transform=UBIRIS_v2_seg_verify_transform)
miche_train_aftersplit = DatasetFromSubset(MICHE_seg_train_dataset, transform=MICHE_seg_verify_transform)
all_datasets_train_aftersplit = DatasetFromSubset(all_datasets_train_dataset, transform=all_datasets_verify_transform)

In [ ]:
all_datasets_train_aftersplit_mean, all_datasets_train_aftersplit_std = getMeanAndStd(all_datasets_train_aftersplit, 3)
print("\n")
print(all_datasets_train_aftersplit_mean)
print(all_datasets_train_aftersplit_std)

4272it [01:17, 55.43it/s]



[ 2.39133161e-10 -1.15617248e-10  3.93928050e-10]
[0.00392157 0.00392157 0.00392157]


In [ ]:
africa_train_aftersplit_mean, africa_train_aftersplit_std = getMeanAndStd(africa_train_aftersplit, 3)
print("\n")
print(africa_train_aftersplit_mean)
print(africa_train_aftersplit_std)

296it [00:10, 28.86it/s]



[5.16373208e-11 5.16373208e-11 5.16373208e-11]
[0.00392157 0.00392157 0.00392157]


In [ ]:
distance_train_aftersplit_mean, distance_train_aftersplit_std = getMeanAndStd(distance_train_aftersplit, 3)
print("\n")
print(distance_train_aftersplit_mean)
print(distance_train_aftersplit_std)

236it [00:06, 35.06it/s]



[-3.29537768e-10 -3.29537768e-10 -3.29537768e-10]
[0.00392157 0.00392157 0.00392157]


In [ ]:
occ_train_aftersplit_mean, occ_train_aftersplit_std = getMeanAndStd(occ_train_aftersplit, 3)
print("\n")
print(occ_train_aftersplit_mean)
print(occ_train_aftersplit_std)

80it [00:02, 38.27it/s]



[-1.14306150e-09  4.72417941e-10 -4.05399238e-10]
[0.00392157 0.00392157 0.00392157]


In [ ]:
off_train_aftersplit_mean, off_train_aftersplit_std = getMeanAndStd(off_train_aftersplit, 3)
print("\n")
print(off_train_aftersplit_mean)
print(off_train_aftersplit_std)

80it [00:02, 36.86it/s]



[-4.47217448e-10  7.51632101e-10 -7.08261687e-10]
[0.00392157 0.00392157 0.00392157]


In [ ]:
m1_train_aftersplit_mean, m1_train_aftersplit_std = getMeanAndStd(m1_train_aftersplit, 3)
print("\n")
print(m1_train_aftersplit_mean)
print(m1_train_aftersplit_std)

1200it [00:26, 45.22it/s]



[3.91123524e-10 3.91123524e-10 3.91123524e-10]
[0.00392157 0.00392157 0.00392157]


In [ ]:
ubiris_train_aftersplit_mean, ubiris_train_aftersplit_std = getMeanAndStd(ubiris_train_aftersplit, 3)
print("\n")
print(ubiris_train_aftersplit_mean)
print(ubiris_train_aftersplit_std)

1356it [00:28, 47.69it/s]



[8.54587714e-11 1.59319778e-10 3.08921783e-10]
[0.00392157 0.00392157 0.00392157]


In [ ]:
miche_train_aftersplit_mean, miche_train_aftersplit_std = getMeanAndStd(miche_train_aftersplit, 3)
print("\n")
print(miche_train_aftersplit_mean)
print(miche_train_aftersplit_std)

544it [00:11, 45.72it/s]



[-2.04364613e-10  2.11635536e-10 -5.66245737e-10]
[0.00392157 0.00392157 0.00392157]


# Image Recognition